<a href="https://colab.research.google.com/github/mit1280/unlimited_classifier/blob/use-llm-with-free-colab/unlimited_classifier_with_phi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the repository
!git clone -b "use-llm-with-free-colab" https://github.com/mit1280/unlimited_classifier.git

# Change to the directory of the cloned repository
%cd unlimited_classifier

Cloning into 'unlimited_classifier'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 117 (delta 56), reused 90 (delta 33), pack-reused 0
Receiving objects: 100% (117/117), 1.08 MiB | 19.72 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/unlimited_classifier


In [ ]:
!pip install -q .

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from unlimited_classifier.scorer import Scorer
from unlimited_classifier import TextClassifier
import torch

Compiling /usr/local/lib/python3.10/dist-packages/unlimited_classifier/labels_trie.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.10/dist-packages/unlimited_classifier/labels_trie.pyx


In [ ]:
model_card = "unsloth/Phi-3-mini-4k-instruct"

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

In [ ]:
classifier = TextClassifier(
    labels=["positive", "negative"],
    model=model_card,
    tokenizer=model_card,
    device= device,
    device_map="auto",
    quantization_config = quantization_config,
    scorer=Scorer()
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset("SetFit/SentEval-CR")

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/3012 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/753 [00:00<?, ? examples/s]

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
import pandas as pd

# Convert Hugging Face Dataset to pandas DataFrame
test_df = dataset['test'].to_pandas()

In [ ]:
def pred_label(example):
    statement = example['text']

    output = classifier.invoke(statement)
    example['pred'] = output
    return example

In [ ]:
test_df = test_df.progress_apply(pred_label, axis=1)

  0%|          | 0/753 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
100%|██████████| 753/753 [16:23<00:00,  1.31s/it]


In [ ]:
# Find the label with the highest confidence
test_df['pred_label'] = test_df['pred'].apply(lambda k: max(k, key=lambda x: x[1])[0])

In [ ]:
def extract_confidences(confidence_scores):
    # Convert list of tuples to dictionary
    scores_dict = dict(confidence_scores)
    # Return positive and negative scores as a Series
    return pd.Series([scores_dict.get('positive', 0), scores_dict.get('negative', 0)], index=['positive', 'negative'])

# Apply the function to the 'confidence_scores' column
test_df[['positive', 'negative']] = test_df['pred'].apply(extract_confidences)


In [ ]:
test_df[test_df['label_text']==test_df['pred_label']].shape[0]/len(test_df)

0.9083665338645418